# Após quatro meses de queda, atendimentos por síndrome gripal nas UPAs em novembro de 2022, até dia 26, quase dobram em relação a outubro
### Quantidade de consultas teve aumento de 96,5% até o último domingo, 27. Ao todo, 67,6% dos atendimentos do mês tiveram classificação de risco verde, considerada pouco urgente

---

Este levantamento foi realizado a partir dos dados fornecidos pela API do IntegraSUS, plataforma da Secretaria da Saúde do Estado do Ceará (Sesa). Eles consideram informações das seguintes Unidades de Pronto Atendimento: 'UPA Autran Nunes', 'UPA Bom Jardim', 'UPA Canindezinho', 'UPA Conjunto Ceará', 'UPA Cristo Redentor', 'UPA Edson Queiroz', 'UPA Itaperi', 'UPA Jangurussu', 'UPA José Walter', 'UPA Messejana', 'UPA Praia Do Futuro', 'UPA Vila Velha'

Em meio a novo crescimento dos casos de Covid-19 no Ceará, é relevante que a imprensa acompanhe os dados oficiais sobre a doença local e nacionalmente.


In [59]:
import pandas as pd
from datetime import date

In [4]:
atendimentos = 'https://integrasus.saude.ce.gov.br/api/acompanhamento-casos-upas/serie-historica-atendimentos?dataInicio=2020-01-01&dataFim=2022-11-26&tipo=&upa=&cid=&classificacaoRisco='

In [5]:
classificacao_risco = 'https://integrasus.saude.ce.gov.br/api/acompanhamento-casos-upas/classificacao-risco?dataInicio=2020-01-01&dataFim=2022-11-26&tipo=&upa=&cid=&classificacaoRisco='

In [6]:
unidades = 'https://integrasus.saude.ce.gov.br/api/acompanhamento-casos-upas/unidade?dataInicio=2020-01-01&dataFim=2022-11-27&tipo=&upa=&cid=&classificacaoRisco='

In [7]:
df_atendimentos = pd.read_json(atendimentos)
df_classificacao = pd.read_json(classificacao_risco)
df_unidades = pd.read_json(unidades)

In [8]:
# salvar arquivos em csv
df_atendimentos.to_csv('bases/originais/df_atendimentos.csv', index=False)
df_classificacao.to_csv('bases/originais/df_classificacao.csv', index=False)
df_unidades.to_csv('bases/originais/df_unidades.csv', index=False)

# Atendimentos

In [9]:
df_atendimentos['dia'] = df_atendimentos['data'].str.split(pat='/', expand=True)[0]
df_atendimentos['mes'] = df_atendimentos['data'].str.split(pat='/', expand=True)[1]
df_atendimentos['ano'] = df_atendimentos['data'].str.split(pat='/', expand=True)[2]

In [10]:
meses = {
    '01': 'janeiro',
    '02': 'fevereiro',
    '03': 'março',
    '04': 'abril',
    '05': 'maio',
    '06': 'junho',
    '07': 'julho',
    '08': 'agosto',
    '09': 'setembro',
    '10': 'outubro',
    '11': 'novembro',
    '12': 'dezembro'
}

In [11]:
atendimentos_mes = df_atendimentos[['mes', 'ano', 'quantidade']].groupby(['mes', 'ano']).sum().reset_index().sort_values(['ano', 'mes'])
atendimentos_mes['ds_mes'] = atendimentos_mes['mes'].replace(meses)

In [12]:
atendimentos_mes = atendimentos_mes[['ano', 'mes', 'ds_mes', 'quantidade']]
atendimentos_mes.tail()

,ano,mes,ds_mes,quantidade
20,2022,07,julho,12763
23,2022,08,agosto,8195
26,2022,09,setembro,7076
29,2022,10,outubro,5921
32,2022,11,novembro,11638


In [13]:
atendimentos_mes.to_csv('bases/arquivos-gerados/atendimentos_mes.csv', index=False)

### atendimento nos três anos

In [14]:
df_atendimentos_mes_ano = atendimentos_mes.pivot(index=['mes', 'ds_mes'], columns='ano', values='quantidade').reset_index()

In [15]:
df_atendimentos_mes_ano.tail()

ano,mes,ds_mes,2020,2021,2022
7,08,agosto,6288.0,9537.0,8195.0
8,09,setembro,4460.0,7862.0,7076.0
9,10,outubro,6452.0,8518.0,5921.0
10,11,novembro,9575.0,6540.0,11638.0
11,12,dezembro,11494.0,27847.0,NaN


In [16]:
df_atendimentos_mes_ano.to_csv('bases/arquivos-gerados/df_atendimentos_mes_ano.csv', index=False)

### diferença entre os meses

In [17]:
atendimentos_mes['diferenca_mes'] = atendimentos_mes['quantidade'].diff()

In [18]:
diferenca_atendimentos_mes_ano = atendimentos_mes.pivot(index=['mes', 'ds_mes'], columns='ano', values='diferenca_mes').reset_index()

In [19]:
diferenca_atendimentos_mes_ano.tail()

ano,mes,ds_mes,2020,2021,2022
7,08,agosto,-917.0,-2406.0,-4568.0
8,09,setembro,-1828.0,-1675.0,-1119.0
9,10,outubro,1992.0,656.0,-1155.0
10,11,novembro,3123.0,-1978.0,5717.0
11,12,dezembro,1919.0,21307.0,NaN


In [20]:
diferenca_atendimentos_mes_ano.to_csv('bases/arquivos-gerados/diferenca_atendimentos_mes_ano.csv', index=False)

### variação percentual entre os meses

In [21]:
atendimentos_mes['mudanca_percentual'] = atendimentos_mes['quantidade'].pct_change()

In [22]:
df_mudanca_percentual = atendimentos_mes.pivot(index=['mes', 'ds_mes'], columns='ano', values='mudanca_percentual').reset_index()

In [23]:
df_mudanca_percentual.tail()

ano,mes,ds_mes,2020,2021,2022
7,08,agosto,-0.127273,-0.201457,-0.357910
8,09,setembro,-0.290712,-0.175632,-0.136547
9,10,outubro,0.446637,0.083439,-0.163228
10,11,novembro,0.484036,-0.232214,0.965546
11,12,dezembro,0.200418,3.257951,NaN


In [24]:
df_mudanca_percentual.to_csv('bases/arquivos-gerados/df_mudanca_percentual.csv', index=False)

### atendimentos 2022

In [25]:
atendimentos_2022 = atendimentos_mes[atendimentos_mes['ano'] == '2022']
atendimentos_2022 = atendimentos_2022[['ds_mes', 'quantidade', 'diferenca_mes', 'mudanca_percentual']]
atendimentos_2022.tail()

,ds_mes,quantidade,diferenca_mes,mudanca_percentual
20,julho,12763,-3594.0,-0.219722
23,agosto,8195,-4568.0,-0.357910
26,setembro,7076,-1119.0,-0.136547
29,outubro,5921,-1155.0,-0.163228
32,novembro,11638,5717.0,0.965546


In [26]:
atendimentos_2022.to_csv('bases/arquivos-gerados/atendimentos_2022.csv', index=False)

In [27]:
df_atendimentos_22 = df_atendimentos[(df_atendimentos['ano'] == '2022')]
df_atendimentos_out_nov_22 = df_atendimentos_22[(df_atendimentos_22['mes'] == '10') | (df_atendimentos_22['mes'] == '11')]

In [56]:
df_atendimentos_out_nov_22 = df_atendimentos_out_nov_22[['data', 'quantidade']]

In [64]:
df_atendimentos_out_nov_22['date'] = pd.to_datetime(df_atendimentos_out_nov_22['data'], format='%d/%m/%Y')

In [74]:
print('max:', df_atendimentos_out_nov_22[df_atendimentos_out_nov_22['date'] < '2022-11-13']['quantidade'].max())
print('media:', round(df_atendimentos_out_nov_22[df_atendimentos_out_nov_22['date'] < '2022-11-13']['quantidade'].mean()))

max: 315
media: 206


In [79]:
print('max:', df_atendimentos_out_nov_22[df_atendimentos_out_nov_22['data'] >= '2022-11-13']['quantidade'].max())
print('media:', round(df_atendimentos_out_nov_22[df_atendimentos_out_nov_22['data'] >= '2022-11-13']['quantidade'].mean()))

max: 1017
media: 418


In [31]:
df_atendimentos_out_nov_22.to_csv('bases/arquivos-gerados/df_atendimentos_out_nov_22.csv', index=False)

# Classificação de risco

In [32]:
df_classificacao['nr_mes'] = df_classificacao['data'].str.split(pat='-', expand=True)[1]

In [33]:
cores = {'BRANCO': 'OUTROS',
         'AZUL': 'OUTROS',
         'VERDE': 'VERDE',
         'AMARELO': 'AMARELO',
         'LARANJA': 'LARANJA',
         'NAO CLASSIFICADO': 'OUTROS',
         'VERMELHO': 'VERMELHO',
         'PRETO': 'OUTROS'}

In [34]:
df_classificacao['nova_classificacaoRisco'] = df_classificacao['classificacaoRisco'].replace(cores)

In [35]:
df_classificacao_mes_ano = df_classificacao[['ano', 'nr_mes', 'nova_classificacaoRisco', 'quantidade']].groupby(['ano', 'nr_mes', 'nova_classificacaoRisco']).sum().reset_index()

In [36]:
df_classificacao_mes_ano['ds_mes'] = df_classificacao_mes_ano['nr_mes'].replace(meses)

In [37]:
df_classificacao_por_mes = df_classificacao_mes_ano.pivot(index=['ano', 'nr_mes', 'ds_mes'], columns='nova_classificacaoRisco', values='quantidade').reset_index().fillna(0)

In [38]:
df_classificacao_por_mes.tail()

nova_classificacaoRisco,ano,nr_mes,ds_mes,AMARELO,LARANJA,OUTROS,VERDE,VERMELHO
30,2022,07,julho,3360,1073,816,7581,74
31,2022,08,agosto,1959,822,485,4979,48
32,2022,09,setembro,1667,707,528,4252,41
33,2022,10,outubro,1478,755,444,3303,73
34,2022,11,novembro,2440,844,749,7654,49


In [39]:
df_classificacao_por_mes.to_csv('bases/arquivos-gerados/df_classificacao_por_mes.csv', index=False)

In [40]:
df_classificacao_por_mes = df_classificacao_por_mes.reset_index()

In [41]:
df_classificacao_por_mes_pct_change = df_classificacao_por_mes[['AMARELO', 'LARANJA', 'OUTROS', 'VERDE', 'VERMELHO']].pct_change()
df_classificacao_por_mes_pct_change = df_classificacao_por_mes_pct_change.apply(lambda x: round(x*100,2)).reset_index()
df_classificacao_por_mes_pct_change = df_classificacao_por_mes_pct_change.rename(columns={
    'AMARELO': 'PCT_AMARELO',
    'LARANJA': 'PCT_LARANJA',
    'OUTROS': 'PCT_OUTROS',
    'VERDE': 'PCT_VERDE',
    'VERMELHO': 'PCT_VERMELHO'
})

In [42]:
df_classificacao_por_mes_com_pct = df_classificacao_por_mes.merge(df_classificacao_por_mes_pct_change, on='index')

In [43]:
df_classificacao_por_mes_com_pct = df_classificacao_por_mes_com_pct[['ano', 'nr_mes', 'ds_mes', 'AMARELO', 'LARANJA', 'OUTROS', 'VERDE', 'VERMELHO', 'PCT_AMARELO', 'PCT_LARANJA', 'PCT_OUTROS', 'PCT_VERDE', 'PCT_VERMELHO']]

In [44]:
df_classificacao_por_mes_com_pct.tail()

nova_classificacaoRisco,ano,nr_mes,ds_mes,AMARELO,LARANJA,OUTROS,VERDE,VERMELHO,PCT_AMARELO,PCT_LARANJA,PCT_OUTROS,PCT_VERDE,PCT_VERMELHO
30,2022,07,julho,3360,1073,816,7581,74,-37.08,-40.59,-17.33,-8.73,39.62
31,2022,08,agosto,1959,822,485,4979,48,-41.70,-23.39,-40.56,-34.32,-35.14
32,2022,09,setembro,1667,707,528,4252,41,-14.91,-13.99,8.87,-14.60,-14.58
33,2022,10,outubro,1478,755,444,3303,73,-11.34,6.79,-15.91,-22.32,78.05
34,2022,11,novembro,2440,844,749,7654,49,65.09,11.79,68.69,131.73,-32.88


In [45]:
df_classificacao_por_mes_com_pct.to_csv('bases/arquivos-gerados/df_classificacao_por_mes_com_pct.csv', index=False)